In [1]:
!nvidia-smi

Thu Jun  1 01:29:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 22%   23C    P8     5W / 250W |  10457MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:B2:00.0 Off |                  N/A |
| 22%   

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set TensorFlow to use the first GPU (assuming you have at least one GPU available)
tf.config.experimental.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[1], 'GPU')

# # Enable memory growth for the GPU
# tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[1], True)

# # Rest of your TensorFlow code...
# ...


2023-06-01 01:29:27.015238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 01:29:29.400389: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/hashimi/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-06-01 01:29:29.400619: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file

Num GPUs Available:  4


In [1]:
import os, sys
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.layers as tfkl
import numpy as np
import matplotlib.pyplot as plt

2023-06-03 12:26:39.238341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 12:26:42.173729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/hashimi/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-06-03 12:26:42.174046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file

In [4]:
# Getting the data and tokenization (character-wise)
with open('shakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    
characters = sorted(list(set(text)))
num_vocab = len(characters)

str_to_idx = {char:idx for idx, char in enumerate(characters)}
idx_to_str = {idx:char for idx, char in enumerate(characters)}

def encoder(string): return [str_to_idx[i] for i in string]
def decoder(idx_list): return ''.join([idx_to_str[i] for i in idx_list])

_data = tf.constant(encoder(text), dtype=tf.int64)
data_split = 0.85
train_data = _data[:int(data_split*_data.shape[0])]
test_data = _data[int(data_split*_data.shape[0]):]

In [5]:
train_data.shape[0], len(_data)

(948084, 1115394)

In [6]:
decoder(train_data[0:15].numpy())

'First Citizen:\n'

In [7]:
# NOTE(Abid): Hyperparameters
batch_size = 64
context_length = 256
learning_rate = 2e-4
embed_dim = 384
num_heads = 6
num_layers = 6
dropout_rate = 0.2
max_iterations = 4_000

In [8]:
def get_next_batch(batch_size, is_train=True):
    data = train_data if is_train else test_data
    rand_idxs = tf.random.uniform(shape=[batch_size],
                                  minval=0,
                                  maxval=(data.shape[0] - context_length),
                                  dtype=tf.int64)
    x_data = tf.concat([
                        tf.expand_dims(data[idx:idx+context_length], axis=0) for idx in rand_idxs
                       ], axis=0)
    y_data = tf.concat([
                        tf.expand_dims(data[idx+1:idx+context_length+1], axis=0) for idx in rand_idxs
                       ], axis=0)
    return x_data, y_data

In [9]:
x, y = get_next_batch(64)
x.shape, y.shape

(TensorShape([64, 256]), TensorShape([64, 256]))

In [10]:
x[0, context_length-5:].numpy(), y[0, context_length-5:].numpy()

(array([32, 20, 33, 25, 14]), array([20, 33, 25, 14, 17]))

$$\displaystyle Attention(Q, K, V) = softmax(\frac{(QK^T)*M_{tril}}{\sqrt{d_k}})V$$

In [18]:
class Head(tfkl.Layer):
    def __init__(self, head_size, context_length, name = "head"):
        super(Head, self).__init__(name=name)
        self.key = tfkl.Dense(head_size, use_bias=False)
        self.query = tfkl.Dense(head_size, use_bias=False)        
        self.value = tfkl.Dense(head_size, use_bias=False)
        self.tril = tf.Variable(tf.cast(
                                        tf.linalg.band_part(tf.ones((context_length, context_length)), -1, 0),
                                        tf.bool
                                        ),
                                trainable=False)
        
        self.dropout = tfkl.Dropout(rate=dropout_rate)
        
    def call(self, inputs):
        """
            Input : (B, T, C)
            
            B : Batch
            T : Token
            C : embed_dim
        """
        _, _, C = inputs.shape
        k = self.key(inputs) # Shape : (B, T, head_size)
        q = self.query(inputs) # Shape : (B, T, head_size)
        v = self.value(inputs) # Shape : (B, T, head_size)
        # NOTE(Abid): Dot product masked self-attention
        weights = q @ tf.transpose(k, perm=[0, 2, 1]) * C**-0.5 # Shape: (B, Wq, Wk)
        weights = tf.where(self.tril, weights, float('-inf'))
        weights = tf.nn.softmax(weights, axis=-1) # Normalize across the query dimension
        weights = self.dropout(weights)
        
        return tf.matmul(weights, v) # (B, Wq, Wk) . (B, T, head_size) -> (B, T, head_size)

In [11]:
 # Checking if the head works
head = Head(10, 7)
ex_input = tf.random.normal((1, 7, 30))
# head(ex_input)
key = head.key(ex_input)
query = head.query(ex_input)
value = head.value(ex_input)
weights = query @ tf.transpose(key, perm=[0, 2, 1]) * 30**-0.5
weights = tf.where(head.tril, weights, float('-inf'))
weights = tf.nn.softmax(weights, axis=-1)
weights = head.dropout(weights)
tf.matmul(weights, value)

<tf.Tensor: shape=(1, 7, 10), dtype=float32, numpy=
array([[[ 2.1266775 , -0.41531247, -2.304038  ,  1.7987556 ,
         -1.6973717 , -0.4803359 ,  0.04099652, -0.24068457,
         -0.20437169,  3.7679799 ],
        [ 0.2817127 , -0.2714178 , -0.36797982,  0.30827332,
         -2.4097033 ,  1.2224154 ,  0.42525762,  1.1447749 ,
          0.6938753 ,  1.5345342 ],
        [ 1.4629328 , -0.57707345, -1.230379  ,  1.1464357 ,
         -1.4589345 ,  0.21226065, -0.20691413,  0.46568716,
          0.19696167,  2.5874577 ],
        [ 0.2317805 , -0.22312945,  0.24044028,  0.35871542,
         -1.5233091 ,  0.94416434,  0.37654433,  1.1855155 ,
          0.42087865,  1.0436945 ],
        [ 1.0228122 , -0.53756744,  0.25492057,  0.5618928 ,
         -0.29603082,  0.5919165 ,  0.10874847,  1.4014163 ,
          0.3468199 ,  0.7130613 ],
        [ 0.64294046,  0.095405  , -0.686411  ,  1.1214666 ,
         -1.4347806 , -0.06384039,  0.43832386,  0.26387972,
         -0.11089922,  1.6661055 ],


In [19]:
class MHA(tfkl.Layer):
    def __init__(self, num_heads, head_size, context_length):
        super(MHA, self).__init__(name="MultiHeadAttention")
        self.heads = [Head(head_size, context_length, "head_"+str(i+1)) for i in range(num_heads)]
        # NOTE(Abid): This projection is used for when the concatenated values are large/smaller
        #             in dimension than embed_dim. If that were to be the case, we wouldn't be able
        #             to add the residual connections
        self.proj = tfkl.Dense(embed_dim)
        self.dropout = tfkl.Dropout(rate=dropout_rate)
        
    def call(self, inputs):
        """
            Input : (B, T, C)
            
            B : Batch
            T : Token
            C : embed_dim
        """
        output = tf.concat([head(inputs) for head in self.heads], axis=-1) # Shape : (B, T, num_head*head_size)
        output = self.proj(output) # Shape : (B, T, C=embed_dim)
        output = self.dropout(output)
        return output

In [20]:
class FeedForward(tfkl.Layer):
    def __init__(self, embed_dim):
        super(FeedForward, self).__init__(name="feedforward")
        self.ff = tf.keras.Sequential([
            tfkl.Dense(4*embed_dim, activation='relu'),
            tfkl.Dense(embed_dim),
            tfkl.Dropout(rate=dropout_rate)
        ])
        
    def call(self, inputs):
        """
            Input : (B, T, C)
            
            B : Batch
            T : Token
            C : embed_dim
        """
        return self.ff(inputs) # Shape : (B, T, C)

In [21]:
class Block(tfkl.Layer):
    def __init__(self, embed_dim, num_head, context_length, name='block'):
        super(Block, self).__init__(name=name)
        head_size = embed_dim // num_head
        self.mha = MHA(num_head, head_size, context_length)
        self.ff = FeedForward(embed_dim)
        self.lnorm_1 = tfkl.LayerNormalization()
        self.lnorm_2 = tfkl.LayerNormalization()
        
    def call(self, inputs):
        """
            Input : (B, T, C)
            
            B : Batch
            T : Token
            C : embed_dim
        """
        output = inputs + self.mha(self.lnorm_1(inputs)) # Shape : (B, T, C)
        return output + self.ff(self.lnorm_2(output))  # Shape : (B, T, C)

In [22]:
class GPT(tf.keras.Model):
    def __init__(self, num_vocab, embed_dim, context_length, num_layers, num_heads):
        super(GPT, self).__init__(name="poor_mans_gpt")
        self.token_embedding = tfkl.Embedding(num_vocab, embed_dim)
        self.positional_embedding = tfkl.Embedding(context_length, embed_dim)
        self.blocks = tf.keras.Sequential([Block(embed_dim, num_heads, context_length, "block_"+str(i+1)) for i in range(num_layers)])
        self.lnorm = tfkl.LayerNormalization()
        self.prob_head = tfkl.Dense(num_vocab)
        
    def call(self, inputs):
        """
            Input : (B, T)
            
            B : Batch
            T : Token
        """
        B, T = inputs.shape
        token_embed = self.token_embedding(inputs) # Shape : (B, T, C=embed_dim)
        position_embed = self.positional_embedding(tf.range(T)) # Shape :(T, C)
        # NOTE(Abid): Broadcast here
        output = token_embed + position_embed # Shape : (B, T, C)
        output = self.blocks(output) # Shape : (B, T, C)
        output = self.lnorm(output)
        probs = self.prob_head(output) # Shape : (B, T, num_vocab)
        return probs

In [23]:
gpt = GPT(num_vocab, embed_dim, context_length, num_layers, num_heads)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = tf.optimizers.Adam(learning_rate=learning_rate)

In [24]:
@tf.function(jit_compile=True)
def train_gpt(x, y):
    with tf.GradientTape() as tape:
        logits = gpt(x)
        loss = loss_fn(y, logits)
    grads = tape.gradient(loss, gpt.trainable_variables)
    optim.apply_gradients(zip(grads, gpt.trainable_variables))
    
    return loss

In [25]:
# Training Loop
iteration = 0
while True:
    if iteration > max_iterations: break
    x, y = get_next_batch(batch_size)
    loss = train_gpt(x, y)
    if iteration % 50 == 0:
        print(f"Iter: {iteration}, Training Loss: {loss}")
    
    iteration += 1

Iter: 0, Training Loss: 4.371750831604004
Iter: 50, Training Loss: 3.2318971157073975
Iter: 100, Training Loss: 2.641303539276123
Iter: 150, Training Loss: 2.5314390659332275
Iter: 200, Training Loss: 2.4863147735595703
Iter: 250, Training Loss: 2.4338483810424805
Iter: 300, Training Loss: 2.4210453033447266
Iter: 350, Training Loss: 2.3864338397979736
Iter: 400, Training Loss: 2.2870495319366455
Iter: 450, Training Loss: 2.202235698699951
Iter: 500, Training Loss: 2.1372435092926025
Iter: 550, Training Loss: 2.084388256072998
Iter: 600, Training Loss: 2.0408549308776855
Iter: 650, Training Loss: 1.9649531841278076
Iter: 700, Training Loss: 1.893986463546753
Iter: 750, Training Loss: 1.8385785818099976
Iter: 800, Training Loss: 1.7673579454421997
Iter: 850, Training Loss: 1.7243809700012207
Iter: 900, Training Loss: 1.6937110424041748
Iter: 950, Training Loss: 1.632707953453064
Iter: 1000, Training Loss: 1.655625343322754
Iter: 1050, Training Loss: 1.6164946556091309
Iter: 1100, Traini

In [26]:
gpt.save('gpt_trained')

INFO:tensorflow:Assets written to: gpt_trained/assets


INFO:tensorflow:Assets written to: gpt_trained/assets


In [29]:
context, _ = get_next_batch(1, False)
print(decoder(context[0].numpy()))

he day we long have looked for:
I am your neighbour, and was suitor first.

TRANIO:
And I am one that love Bianca more
Than words can witness, or your thoughts can guess.

GREMIO:
Youngling, thou canst not love so dear as I.

TRANIO:
Graybeard, thy love do


In [30]:
def generate_text(context, max_generated_tokens):
    """
        input : (B, T)
    
        B : Batch
        T : Token
    """
    print(decoder(context[0].numpy()), end='')
    word_segment = ''
    for _ in range(max_generated_tokens):
        current_context = context[:, -context_length:] # Shape : (B, T)
        logits = gpt(current_context) # Shape : (B, T, num_vocab)
        # NOTE(Abid): We only care about the prediction of the last token
        logits = logits[:, -1, :] # Shape : (B, 1, num_vocab)
        # NOTE(Abid): It seems random.categorical requires logits, so no need for softmax here.
        next_token = tf.random.categorical(tf.expand_dims(logits[0], axis=0), 1)
        context = tf.concat([context, next_token], axis=1)
        word_segment += decoder(next_token[0].numpy())
        if next_token[0][0] == 1 or next_token[0][0] == 0:
            print(word_segment, end='')
            word_segment = ''

In [31]:
generate_text(context, 1_000)

he day we long have looked for:
I am your neighbour, and was suitor first.

TRANIO:
And I am one that love Bianca more
Than words can witness, or your thoughts can guess.

GREMIO:
Youngling, thou canst not love so dear as I.

TRANIO:
Graybeard, thy love down,--

ROVERS:
Those brother, my gracious boy:
For, by his crown unto looking apt;
And leason to grates it in me, I'll bear
My cimissih'd to thee measure from whom;
But do good from her with issue. My Leonte,
It was for me. I most your news friends
Hast accomplish'd to the courtesy,
Would I lent but my wedded to destroy;
So so I have bound whom I swear keeps
Serving you for
The counters, if you'r lace way, never York,
Not doth deny, such as your guiders
More past me, your pardon
Give other twain, to spoir wearing,
When sureon'd the seizes of sevented!

Second Citizen:
Once that, I know my name is a duty flesh:
'Tis virtue way
I'ld not by make melaces. I pray you,
I the wretches not so so shrill.

Nurse:
The queen shrews: let's not tha

In [2]:
tf.__version__

'2.11.0'